# The Sopranos
### Рейтинг серий и режиссеры

In [1]:
import pandas as pd  # Для работы с табличными данными

import seaborn as sns  # Для визуализации данных

import time  # Для работы с временными задержками

import requests  # Для HTTP-запросов
from bs4 import BeautifulSoup  # Для парсинга HTML

%matplotlib inline  # Для отображения графиков в Jupyter

In [2]:
def get_sopranos_ratings():
    
    """Функция для сбора рейтингов эпизодов сериала 'Клан Сопрано' с IMDB"""
    
    base_url = "https://www.imdb.com/title/tt0141842/episodes"  # Базовый URL
    seasons = []# Список для хранения данных
    
    # Проходим по всем сезонам (от 1 до 6)
    for season in range(1, 7):
        
        # Формируем URL для конкретного сезона
        url = f"{base_url}?season={season}"
        
        # Заголовки для имитации браузера и избежания блокировки
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
            "Accept-Language": "en-US,en;q=0.9"
        }
        
        # Отправляем GET-запрос и получаем ответ
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'lxml')
        
        # Находим все контейнеры с эпизодами
        episode_containers = soup.find_all('article', class_='sc-fcf4d924-1 emwOvP episode-item-wrapper')
        
        # Обрабатываем каждый эпизод
        for episode in episode_containers:
            
            # Извлекаем заголовок эпизода
            title = episode.find('div', class_='ipc-title__text').text.strip()
            
            # Парсим номер эпизода (формат "S01.E01 Название")
            episode_number = title.split('.E')[1].split(' ')[0]
            
            # Извлекаем название эпизода
            ep_title = title.split(' ∙ ')[1]
            
            # Получаем рейтинг эпизода
            rating = episode.find('span', class_='ipc-rating-star--rating').text.strip()
            
            # Добавляем данные в список
            seasons.append({
                'season': season,
                'ep_num': episode_number,
                'ep_title': ep_title,
                'ep_rating': rating
            })

    # Возвращаем собранные данные в виде DataFrame
    return pd.DataFrame(seasons)

In [3]:
# Сохраняем рейтинги в CSV-файл

# get_sopranos_ratings().to_csv('sopranos_ratings.csv', index=False)

In [6]:
# Загружаем данные из CSV-файлов

sprns_rat_df = pd.read_csv('sopranos_ratings.csv')

In [4]:
sprns_dir_df = pd.read_csv('sopranos_directors.csv')

In [8]:
# Просмотр первых строк таблицы с рейтингами

sprns_rat_df.head()

,season,ep_num,ep_title,ep_rating
0,1,1,Pilot,8.4
1,1,2,46 Long,8.2
2,1,3,"Denial, Anger, Acceptance",8.5
3,1,4,Meadowlands,8.5
4,1,5,College,8.9


In [17]:
# Обработка данных о режиссерах
# Извлекаем номер сезона из строки формата "S01E01":

sprns_dir_df['season'] = sprns_dir_df.season_ep.str.split('.').str[0].str[1]

In [23]:
# Создаем комбинированный столбец "сезон.эпизод"

sprns_dir_df['ep_num'] = (sprns_dir_df.season_ep.str.split('.').str[1].str[1:]).astype('int')

In [31]:
# Создаем комбинированный столбец "сезон.эпизод"

sprns_dir_df['season_ep_num'] = sprns_dir_df.season + '.' + sprns_dir_df.ep_num.astype(str)

In [36]:
# Удаляем исходный столбец 

sprns_dir_df.drop(columns=['season_ep'], inplace=True)

In [38]:
# Просмотр информации о DataFrame с режиссерами

sprns_dir_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   director       86 non-null     object
 1   season         86 non-null     object
 2   ep_num         86 non-null     int64 
 3   season_ep_num  86 non-null     object
dtypes: int64(1), object(3)
memory usage: 2.8+ KB


In [40]:
# Сохраняем обработанные данные о режиссерах

# sprns_dir_df.to_csv('sopranos_directors_edited.csv', index=False)